In [7]:
import os
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

input_root = "nail_disease_dataset"  
output_root = "preprocessed_dataset" 
target_size = (224, 224)  # Resize dimensions

In [3]:
# CLAHE 
def apply_clahe(img):
    """
    Enhances local contrast using CLAHE on the L-channel in LAB color space.
    """
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    enhanced_img = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)
    return enhanced_img

In [4]:
# Bilateral Filtering
def apply_bilateral_filter(img):
    """
    Applies bilateral filtering to reduce noise while preserving edges.
    """
    return cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)

In [5]:
def preprocess_and_save_images(input_dir, output_dir):
    """
    Loops through the class folders in input_dir (train/test),
    applies preprocessing steps, and saves the processed images to output_dir.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_name in os.listdir(input_dir):
        class_input_path = os.path.join(input_dir, class_name)
        class_output_path = os.path.join(output_dir, class_name)

        if not os.path.exists(class_output_path):
            os.makedirs(class_output_path)

        for image_name in tqdm(os.listdir(class_input_path), desc=f"Processing {class_name}"):
            image_path = os.path.join(class_input_path, image_name)
            try:
                # Read and resize image
                img = cv2.imread(image_path)
                img = cv2.resize(img, target_size)

                # Apply preprocessing
                img = apply_clahe(img)
                img = apply_bilateral_filter(img)

                # Normalize image
                img = img.astype(np.float32) / 255.0

                # Save as .png
                output_image_path = os.path.join(class_output_path, image_name)
                cv2.imwrite(output_image_path, (img * 255).astype(np.uint8))

            except Exception as e:
                print(f" Error processing {image_path}: {e}")


In [8]:
#  preprocessing on both train and test folders
for split in ["train", "test"]:
    input_dir = os.path.join(input_root, split)
    output_dir = os.path.join(output_root, split)
    preprocess_and_save_images(input_dir, output_dir)

Processing psoriasis:  85%|████████▍ | 78/92 [00:01<00:00, 45.97it/s]

 Error processing nail_disease_dataset\test\psoriasis\psoriatic-nail-037__WatermarkedWyJXYXRlcm1hcmtlZCJd.jfif: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:836: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'cv::imwrite_'



Processing psoriasis: 100%|██████████| 92/92 [00:02<00:00, 42.64it/s]
